In [1]:
from __future__ import annotations

import jax
import jax.numpy as jnp

import hodel.dismech as dismech

jax.config.update("jax_enable_x64", True)

geom = dismech.Geometry(
    rod_r0=0.001,
    shell_h=0,
)

mat = dismech.Material(
    density=1200,
    youngs_rod=2e6,
    youngs_shell=0,
    poisson_rod=0.5,
    poisson_shell=0,
)

"""# TODO: Figure out how sim params works for HoDEL
sim_params = dismech.SimParams(
    max_iter=30,
    tol=1e-4,
    ftol=1e-4,
    dtol=1e-2,
)"""

mesh = dismech.Mesh.from_txt("rod.txt")
top, state, mass, theta, triplets = dismech.from_legacy_custom(mesh, geom, mat)

In [2]:
from dataclasses import dataclass
from jax.tree_util import register_dataclass

import jaxtyping

import hodel


def get_indices(
    q: jax.Array,
    top: dismech.Connectivity,
    fixed_nodes: jax.Array | None = None,
    fixed_edges: jax.Array | None = None,
) -> tuple[jax.Array, jax.Array]:
    nodes = (
        jnp.array([]) if fixed_nodes is None else dismech.map_node_to_dof(fixed_nodes)
    )
    edges = jnp.array([]) if fixed_edges is None else top.edge_dofs[0] + fixed_edges
    idx_b = jnp.union1d(nodes, edges)
    return idx_b, jnp.setdiff1d(jnp.arange(q.shape[0]), idx_b)


idx_b, idx_f = get_indices(
    state.q, top, jnp.array([0, 1, 2, 18, 19, 20]), jnp.array([0, 1, 18, 19])
)


@register_dataclass
@dataclass(frozen=True)
class TripletAux:
    """parametrized external force."""

    top: dismech.Connectivity  # for state.update()
    idx_f: jax.Array
    idx_b: jax.Array


def get_gravity(mass: jax.Array, g: jax.Array = jnp.array([0.0, 0.0, -9.81])):
    return mass * jnp.concat(
        [
            jnp.tile(g, mesh.nodes.shape[0]),
            jnp.zeros(mesh.edges.shape[0]),
        ]
    )


def get_W(lambda_: jax.Array, aux: TripletAux):
    """Just gravity."""
    return get_gravity(mass)[aux.idx_f]


def fixed_0(lambda_: jax.Array, aux: TripletAux) -> jax.Array:
    """Compress and contort."""
    return state.q[aux.idx_b] + lambda_ * jnp.concat(
        [
            jnp.array([0.02, 0.0, 0, 0.02, 0.0, 0, 0.02, 0.0, 0]),
            jnp.zeros(aux.idx_b.shape[0] - 13),
            jnp.array([1.0, 1.0, 0.0, 0.0]),
        ]
    )


def get_q(xf: jax.Array, xb: jax.Array, aux: TripletAux) -> jax.Array:
    """Helper to construct q from xf and xb."""
    q = jnp.empty((aux.idx_f.shape[0] + aux.idx_b.shape[0]), xf.dtype)
    return q.at[aux.idx_f].set(xf).at[aux.idx_b].set(xb)


def update_state(
    xf: jax.Array, xb: jax.Array, aux: TripletAux, carry: dismech.StaticState
) -> dismech.StaticState:
    """Construct q from xf and xb and update state."""
    q = get_q(xf, xb, aux)
    carry_new = carry.update(q, aux.top)
    return carry_new


def get_batch_energy(
    xf: jax.Array,
    xb: jax.Array,
    Theta: jaxtyping.PyTree,
    aux: TripletAux,
    carry: dismech.StaticState,
) -> jax.Array:
    q = get_q(xf, xb, aux)
    state = carry.update(q, aux.top)
    return jnp.sum(jax.vmap(lambda t: t.get_energy(state, Theta))(triplets))


xf0 = state.q[idx_f]
aux = TripletAux(top, idx_f, idx_b)
sim = hodel.HODEL(
    get_batch_energy, get_W_fn=get_W, get_xb_fn=fixed_0, carry_fn=update_state
)

In [3]:
lambdas = jnp.linspace(0, 1.0, 100)
xf_star = sim.sim(lambdas, xf0, theta, aux, state, nsteps=10)
xb_star = jax.vmap(fixed_0, (0, None))(lambdas, aux)
qs = jax.vmap(get_q, (0, 0, None))(xf_star, xb_star, aux)
dismech.animate(lambdas, qs, top)

err: 6.046404034766759e-14
err: 9.608656856009097e-14
err: 6.188873908892259e-14
err: 5.4150782345462984e-14
err: 5.971066300394765e-14
err: 8.270415868007848e-14
err: 6.895960986287744e-14
err: 6.694768901843761e-14
err: 7.228298388095796e-14
err: 6.563548917072589e-14
err: 3.8180994252211976e-14
err: 6.220395950659139e-14
err: 4.406758947710245e-14
err: 4.709366727015592e-14
err: 5.425724139181742e-14
err: 6.381452729765357e-14
err: 3.216293852114972e-14
err: 4.2972880534593514e-14
err: 5.721973869745297e-14
err: 5.6036231044776184e-14
err: 6.913615928874476e-14
err: 3.4755034787759503e-14
err: 6.937467949624854e-14
err: 4.934692445810657e-14
err: 5.155491090087993e-14
err: 8.25310789220385e-14
err: 5.34180278013622e-14
err: 4.141545183855711e-14
err: 8.347645414081148e-14
err: 2.8597148578709214e-14
err: 4.721270091946107e-14
err: 6.71826248323458e-14
err: 5.893708537282264e-14
err: 6.139890970938936e-14
err: 4.7124825813368776e-14
err: 4.2410106461130774e-14
err: 7.962069950398944e

In [7]:
"""
lambdas = jnp.linspace(0, 1, 10)
xf_stars = sim.sim(lambdas, xf0, theta, aux, state)
key = jax.random.PRNGKey(0)
Theta0 = jax.random.uniform(key, [5], minval=1e-1, maxval=1e1)
lr = 1e-2
nepochs = 1



final_Theta, L = sim.batch_learn(
    lambdas,
    xf0,
    xf_stars,
    Theta0,
    batch_aux,
    batch_size=2,
    optim=optax.adam(lr),
    nepochs=nepochs,
)

print("True Theta:", theta)
print("Theta0:", Theta0)
print("Final Theta:", final_Theta)"""
print(theta)

[6.28318531e+00 6.28318531e+00 1.57079633e-06 1.57079633e-06
 1.04719755e-06]


In [14]:
key = jax.random.PRNGKey(0)
Theta0 = jax.random.uniform(key, [5], minval=1e-1, maxval=1e1)
print(Theta0)

[4.24272541 2.241325   9.65668247 5.78755528 5.36904223]


In [22]:
lambdas = jnp.linspace(0, 1.0, 100)
theta0 = jnp.array([6.28e-1, 6.28e-1, 1.57e-6, 1.57e-6, 1.04e-6])
xf_star = sim.sim(lambdas, xf0, theta0, aux, state, nsteps=10)
xb_star = jax.vmap(fixed_0, (0, None))(lambdas, aux)
qs = jax.vmap(get_q, (0, 0, None))(xf_star, xb_star, aux)
dismech.animate(lambdas, qs, top)

err: 4.048049085488149e-15
err: 6.427877052588902e-15
err: 4.6439361632553254e-15
err: 5.508836888010558e-15
err: 3.910850930539888e-15
err: 5.45722412924611e-15
err: 6.628639924189273e-15
err: 6.673603459997375e-15
err: 4.016882797384843e-15
err: 7.468320427562303e-15
err: 4.829249015849272e-15
err: 8.199051519197987e-15
err: 5.382421625646579e-15
err: 5.826456087926296e-15
err: 6.633394233911186e-15
err: 5.771388888741729e-15
err: 3.558455516591744e-15
err: 4.7231039601971425e-15
err: 3.706661839068774e-15
err: 7.698952434765615e-15
err: 4.488436083018028e-15
err: 2.971429337722291e-15
err: 6.524080230409277e-15
err: 4.313171423439484e-15
err: 5.75619312487291e-15
err: 6.7915453987895684e-15
err: 7.056238623307199e-15
err: 5.408565901070943e-15
err: 6.352619580307701e-15
err: 4.7391576921840615e-15
err: 4.747445814855249e-15
err: 6.647769067939668e-15
err: 7.415468883158468e-15
err: 4.0705378823742896e-15
err: 4.908474170622908e-15
err: 4.786612016637371e-15
err: 3.2856017970574838e-